In [67]:
import tweepy as tw
import pandas as pd
import configparser as cp
from textblob import TextBlob
import re
import palavras
import json


In [68]:
config = cp.ConfigParser()
config.read('config.ini')
api_key = config['twitter_credenciais']['api_key']
api_key_secret = config['twitter_credenciais']['api_key_secret']
access_token = config['twitter_credenciais']['access_token']
access_token_secret = config['twitter_credenciais']['access_token_secret']

In [69]:
auth = tw.OAuthHandler(api_key,api_key_secret)
auth.set_access_token(access_token,access_token_secret)
api = tw.API(auth)

In [70]:
df_tweets_positivos = pd.DataFrame()
df_tweets_negativos = pd.DataFrame()
tweets_cursor_positivos = tw.Cursor(api.search_tweets, q=palavras.positivas, tweet_mode="extended", lang="en").items(250)
tweets_cursor_negativos = tw.Cursor(api.search_tweets, q=palavras.negativas, tweet_mode="extended", lang="en").items(250)

In [71]:
# public_tweets = api.home_timeline()
# for i in public_tweets:
#     print(i._json.keys())

In [72]:
def coletar_tweets_positivos(tweets_cursor_positivos):
    tweets_positivos = []
    global df_tweets_positivos
    for i in tweets_cursor_positivos:
        tweet_id = i.id_str
        tweet_text = i.full_text
        tweet_rts = i.retweet_count
        tweet_favs = i.favorite_count
        tweet_date = i.created_at
        #tweet_lang = ''
        tweet_positivos = {
            'id': tweet_id,
            'text': tweet_text,
            'number_of_retweets': tweet_rts,
            'number_of_favorites': tweet_favs,
            'date': tweet_date,
            #'language': tweet_lang,
            'user_id': i.user.id,
            'user_screen_name': i.user.screen_name
        }
        tweets_positivos.append(tweet_positivos)
    df_tweets_positivos = df_tweets_positivos.append(tweets_positivos, ignore_index=True)

In [73]:
def coletar_tweets_negativos(tweets_cursor_negativos):
    tweets_negativos = []
    global df_tweets_negativos
    for i in tweets_cursor_negativos:
        tweet_id = i.id_str
        tweet_text = i.full_text
        tweet_rts = i.retweet_count
        tweet_favs = i.favorite_count
        tweet_date = i.created_at
        #tweet_lang = ''
        tweet_negativos = {
            'id': tweet_id,
            'text': tweet_text,
            'number_of_retweets': tweet_rts,
            'number_of_favorites': tweet_favs,
            'date': tweet_date,
            #'language': tweet_lang,
            'user_id': i.user.id,
            'user_screen_name': i.user.screen_name
        }
        tweets_negativos.append(tweet_negativos)
    df_tweets_negativos = df_tweets_negativos.append(tweets_negativos, ignore_index=True)

In [74]:
extracao_positivos = coletar_tweets_positivos(tweets_cursor_positivos);
df_tweets_positivos.to_csv('tweets-positivos.csv')
extracao_negativos = coletar_tweets_negativos(tweets_cursor_negativos);
df_tweets_negativos.to_csv('tweets-negativos.csv')

C:\Users\vceesar\AppData\Local\Temp\ipykernel_8784\1746576647.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tweets_positivos = df_tweets_positivos.append(tweets_positivos, ignore_index=True)
C:\Users\vceesar\AppData\Local\Temp\ipykernel_8784\3010794071.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tweets_negativos = df_tweets_negativos.append(tweets_negativos, ignore_index=True)


In [75]:
frames = [df_tweets_positivos,df_tweets_negativos]
df = pd.concat(frames)
df =  df.rename({'text':'Tweets'}, axis = 1)


In [76]:
def limpar_tweets(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'@[A-Za-z0-9]_+', '', text)
    text = re.sub(r'@_[A-Za-z0-9]+', '', text)
    text = re.sub(r'@_[A-Za-z0-9]_+', '', text)# Remove mencoes
    text = re.sub(r'#','', text) # Remove simbolo de hashtags
    text = re.sub(r'RT[\s]+','',text) # Remove Retweets
    text = re.sub(r'https?:/\/\S+','',text) # Remove URLS

    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'', text)
    return text

In [77]:
df['Tweets'] = df['Tweets'].apply(limpar_tweets)

In [78]:
def extrair_subjetividade(text):
    return TextBlob(text).sentiment.subjectivity

def extrair_polaridade(text):
    return TextBlob(text).sentiment.polarity

In [79]:
df['Subjetividade'] = df['Tweets'].apply(extrair_subjetividade)
df['Polaridade'] = df['Tweets'].apply(extrair_polaridade)

In [80]:
pd.set_option('display.max_rows', df.shape[0]+1)
len(df.index)

250

In [81]:
df = df.drop(df[df['Tweets'] == ''].index)
len(df.index)

250

In [82]:
def analisarTweet(a):
    if a<0:
        return 'Negativo'
    elif a ==0:
        return 'Neutro'
    else:
        return 'Positivo'

In [83]:
df['Score'] = df['Polaridade'].apply(analisarTweet)

In [84]:
# df.head(50)

,id,Tweets,number_of_retweets,number_of_favorites,date,user_id,user_screen_name,Subjetividade,Polaridade,Score
0,1514733964023504910,FELIZ CUMPLEEEEEEE FASTY!!!!!!!!!!,0,0,2022-04-14 22:34:54+00:00,1391118726343413772,Vadark7,0.000000,0.000000,Neutro
1,1514733771253293057,QUACKITY RE FELIZ,0,0,2022-04-14 22:34:08+00:00,1464662218113531910,cookiecoquette,0.000000,0.000000,Neutro
2,1514733658543996938,Listen to Alejandra Feliz - Seis Nueve Ft. Jay...,0,0,2022-04-14 22:33:41+00:00,1175173988898738176,BachataSensualR,0.000000,0.000000,Neutro
3,1514733615682396165,a ivy comprou daisy jones = eu mt feliz,0,0,2022-04-14 22:33:31+00:00,1320841826530635778,basketbdiaries,0.000000,0.000000,Neutro
4,1514732322708328448,_ OMG FELIZ CUMPLEEEE&lt;33,0,0,2022-04-14 22:28:22+00:00,1297661204576403456,minho_ranita,0.000000,0.000000,Neutro
5,1514732149810905097,feliz halloween,0,1,2022-04-14 22:27:41+00:00,1030282591289655297,GValcado,0.000000,0.000000,Neutro
6,1514731855957970953,Feliz cumpleaños \nLove you a lot now you can...,1,3,2022-04-14 22:26:31+00:00,844996275355500545,echo_silverdoll,0.600000,1.000000,Positivo
7,1514731744506880001,Fallin’\nFree\nFor you\nFriends\n(Feliz Navid...,0,1,2022-04-14 22:26:04+00:00,866029067673505798,FridaArildslund,0.800000,0.400000,Positivo
8,1514731687305007109,"_Regueiro - Bounas nouches. ""FELIS SEXTA-FEIRA...",0,0,2022-04-14 22:25:51+00:00,342197984,juanteio,0.733333,0.733333,Positivo
9,1514731640429465602,"_ - Bounas nouches. ""FELIS SEXTA-FEIRA SANTA""....",0,0,2022-04-14 22:25:40+00:00,342197984,juanteio,0.733333,0.733333,Positivo
